In [14]:
import gc
import os
import random
import shutil
import sys
import time

import argparse
from collections import OrderedDict
from pathlib import Path
from tqdm.auto import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning import _logger as log
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TestTubeLogger

import ogb
import torch_geometric as tg

import matplotlib.pyplot as plt

from ogb.utils import smiles2graph
from ogb.lsc import PygPCQM4MDataset, PCQM4MEvaluator

In [7]:
ROOT = '../../data'

In [8]:
pyg_dataset = PygPCQM4MDataset(root = ROOT, smiles2graph = smiles2graph)

In [13]:
pyg_dataset.get_idx_split()

{'train': tensor([      0,       1,       2,  ..., 3045357, 3045358, 3045359]),
 'valid': tensor([3045360, 3045361, 3045362,  ..., 3426027, 3426028, 3426029]),
 'test': tensor([3426030, 3426031, 3426032,  ..., 3803450, 3803451, 3803452])}

In [18]:
??PCQM4MEvaluator

Init signature: PCQM4MEvaluator()
Docstring:      <no docstring>
Source:        
class PCQM4MEvaluator:
    def __init__(self):
        '''
            Evaluator for the PCQM4M dataset
            Metric is Mean Absolute Error
        '''
        pass 

    def eval(self, input_dict):
        '''
            y_true: numpy.ndarray or torch.Tensor of shape (num_graphs,)
            y_pred: numpy.ndarray or torch.Tensor of shape (num_graphs,)
            y_true and y_pred need to be of the same type (either numpy.ndarray or torch.Tensor)
        '''
        assert('y_pred' in input_dict)
        assert('y_true' in input_dict)

        y_pred, y_true = input_dict['y_pred'], input_dict['y_true']

        assert((isinstance(y_true, np.ndarray) and isinstance(y_pred, np.ndarray))
                or
                (isinstance(y_true, torch.Tensor) and isinstance(y_pred, torch.Tensor)))
        assert(y_true.shape == y_pred.shape)
        assert(len(y_true.shape) == 1)

        if isinstance(y